In [130]:
import pandas as pd
import numpy as np

In [136]:
df1 = pd.read_csv("Underlying Cause of Death, 2006.txt",  delimiter="\t")
df2 = pd.read_csv("Underlying Cause of Death, 2007.txt",  delimiter="\t")
df3 = pd.read_csv("Underlying Cause of Death, 2008.txt",  delimiter="\t")
df4 = pd.read_csv("Underlying Cause of Death, 2009.txt",  delimiter="\t")
df5 = pd.read_csv("Underlying Cause of Death, 2010.txt",  delimiter="\t")
df6 = pd.read_csv("Underlying Cause of Death, 2011.txt",  delimiter="\t")
df7 = pd.read_csv("Underlying Cause of Death, 2012.txt",  delimiter="\t")
df8 = pd.read_csv("Underlying Cause of Death, 2013.txt",  delimiter="\t")
df9 = pd.read_csv("Underlying Cause of Death, 2014.txt",  delimiter="\t")
df10 = pd.read_csv("Underlying Cause of Death, 2015.txt",  delimiter="\t")

In [138]:
df=[df1,df2,df3, df4, df5, df6, df7, df8, df9, df10]
df_new=pd.concat(df)

In [139]:
temp=df_new['County'].str.split(",",expand=True)
df_new['County']=temp[0]
df_new['State']=temp[1]

In [140]:
df_new['Drug/Alcohol Induced Cause'].unique()

array(['All other non-drug and non-alcohol causes',
       'Drug poisonings (overdose) Unintentional (X40-X44)',
       'All other alcohol-induced causes',
       'All other drug-induced causes',
       'Drug poisonings (overdose) Undetermined (Y10-Y14)',
       'Drug poisonings (overdose) Suicide (X60-X64)', nan,
       'Alcohol poisonings (overdose) (X45, X65, Y15)',
       'Drug poisonings (overdose) Homicide (X85)'], dtype=object)

In [142]:
df_death=df_new.loc[df_new['Drug/Alcohol Induced Cause'].isin(['Drug poisonings (overdose) Unintentional (X40-X44)', 'Drug poisonings (overdose) Undetermined (Y10-Y14)', 'Drug poisonings (overdose) Suicide (X60-X64)'])]

In [143]:
columnsTitles = ['Notes', 'State', 'County', 'County Code', 'Year', 'Year Code',
       'Drug/Alcohol Induced Cause', 'Drug/Alcohol Induced Cause Code',
       'Deaths']

df_death = df_death.reindex(columns=columnsTitles).copy()

In [148]:
df_death=df_death.drop(["Notes", "Year Code", "Drug/Alcohol Induced Cause Code"],axis=1 )

In [151]:
df_death.head()

,State,County,County Code,Year,Drug/Alcohol Induced Cause,Deaths
1,AL,Baldwin County,1003.0,2006.0,Drug poisonings (overdose) Unintentional (X40-...,11
12,AL,Chilton County,1021.0,2006.0,Drug poisonings (overdose) Unintentional (X40-...,13
39,AL,Jefferson County,1073.0,2006.0,Drug poisonings (overdose) Unintentional (X40-...,55
55,AL,Mobile County,1097.0,2006.0,Drug poisonings (overdose) Unintentional (X40-...,23
56,AL,Mobile County,1097.0,2006.0,Drug poisonings (overdose) Undetermined (Y10-Y14),10


In [167]:
df_death['County Code']=df_death['County Code'].astype(int)
df_death['Year']=df_death['Year'].astype(int)

In [170]:
df_death.head()

,State,County,County Code,Year,Drug/Alcohol Induced Cause,Deaths
1,AL,Baldwin County,1003,2006,Drug poisonings (overdose) Unintentional (X40-...,11
12,AL,Chilton County,1021,2006,Drug poisonings (overdose) Unintentional (X40-...,13
39,AL,Jefferson County,1073,2006,Drug poisonings (overdose) Unintentional (X40-...,55
55,AL,Mobile County,1097,2006,Drug poisonings (overdose) Unintentional (X40-...,23
56,AL,Mobile County,1097,2006,Drug poisonings (overdose) Undetermined (Y10-Y14),10
